In [10]:
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import parse_netmhcpan_header
df = pd.read_csv('../29165_NetMHCpan.xls', header=[0,1], sep = '\t')
df = df.drop(index = df.sample(3).index)
df.columns = parse_netmhcpan_header(df.columns)
pep = pd.read_csv('../output/test_pep/9mers_Human_split_0.pep', header = None)
pep.columns = ['Peptide']


In [2]:
display(df.head(5))
display(pep.head(5))

base                     HLA-A01:01                                        \
   Pos    Peptide       ID       core      icore EL-score  EL_Rank BA-score   
0    0  MSSHEGGKK  PEPLIST  MSSHEGGKK  MSSHEGGKK   0.0103   4.8079   0.1270   
1    0  SSHEGGKKK  PEPLIST  SSHEGGKKK  SSHEGGKKK   0.0052   7.0502   0.0383   
2    0  SHEGGKKKA  PEPLIST  SHEGGKKKA  SHEGGKKKA   0.0002  41.1667   0.0133   
3    0  HEGGKKKAL  PEPLIST  HEGGKKKAL  HEGGKKKAL   0.0006  24.0652   0.0120   
4    0  EGGKKKALK  PEPLIST  EGGKKKALK  EGGKKKALK   0.0002  39.9231   0.0191   

           HLA-A03:01  ...                   HLA-A26:01                      \
   BA_Rank       core  ... BA-score  BA_Rank       core      icore EL-score   
0   4.2039  MSSHEGGKK  ...   0.3113   2.3594  MSSHEGGKK  MSSHEGGKK   0.0009   
1  37.4149  SSHEGGKKK  ...   0.2516   3.4521  SSHEGGKKK  SSHEGGKKK   0.0027   
2  89.0327  SHEGGKKKA  ...   0.0122  91.3582  SHEGGKKKA  SHEGGKKKA   0.0000   
3  91.3973  HEGGKKKAL  ...   0.0091  95.4398  HEGGKKKAL  HEGGKKKAL   0.0010   
4  75.5713  EGGKKKALK  ...   0.0564  28.0659  EGGKKKALK  EGGKKKALK   0.0001   

                                 end     
   EL_Rank BA-score  BA_Rank     Ave NB  
0  15.5512   0.0628  12.3067  0.0289  0  
1   9.2159   0.0336  29.0975  0.0970  1  
2  57.1429   0.0068  92.4858  0.0001  0  
3  14.6818   0.0165  60.3989  0.0006  0  
4  42.4000   0.0128  72.1920  0.0006  0  

[5 rows x 23 columns]

,Peptide
0,MSSHEGGKK
1,SSHEGGKKK
2,SHEGGKKKA
3,HEGGKKKAL
4,EGGKKKALK


In [11]:
len(df), len(pep)

(8, 11)

In [12]:
df.set_index(('base', 'Peptide'), inplace=True)
df.index.name = 'Peptide'

In [20]:
hlas = set([x for x in df.columns.get_level_values(0) if 'hla' in x.lower()])
ranks = [x for x in df.columns if x[0] in hlas and x[1] == 'EL_Rank' or x[1] == 'BA_Rank']
out = pd.merge(df[ranks].idxmin(axis=1).apply(lambda x: x[0]).rename('HLA'), df[ranks].min(axis=1).rename('xd'), left_index = True, right_index=True)

In [21]:
def return_columns(row, df):
    return [x for x in df.columns if x[0] == row['HLA']]

In [22]:
filtered = out.apply(lambda x: df.loc[x.name, return_columns(x, df)].values, axis=1)

In [ ]:
vals = pd.DataFrame.from_dict(dict(zip(filtered.index, filtered.values))).T
vals.index.name = 'Peptide'
vals.columns = ['core', 'icore', 'EL_score', 'EL_rank', 'BA_score', 'BA_rank']
vals

,core,icore,EL_score,EL_rank,BA_score,BA_rank
Peptide,,,,,,
MSSHEGGKK,MSSHEGGKK,MSSHEGGKK,0.0754,2.2719,0.3113,2.3594
SHEGGKKKA,SHEGGKKKA,SHEGGKKKA,0.0002,41.1667,0.0133,89.0327
HEGGKKKAL,HEGGKKKAL,HEGGKKKAL,0.001,14.6818,0.0165,60.3989
EGGKKKALK,EGGKKKALK,EGGKKKALK,0.0014,12.034,0.0564,28.0659
GGKKKALKQ,GGKKKALKQ,GGKKKALKQ,0.0002,24.48,0.0244,67.2066
GKKKALKQP,GKKKALKQP,GKKKALKQP,0.0,68.75,0.0063,96.819
KKKALKQPK,KKKALKQPK,KKKALKQPK,0.0055,7.1007,0.0838,16.5062
ALKQPKKQA,ALKQPKKQA,ALKQPKKQA,0.0034,8.6334,0.0479,34.5175


In [27]:
out.drop(columns=['xd']).merge(vals, left_index=True, right_index=True)

,HLA,core,icore,EL_score,EL_rank,BA_score,BA_rank
Peptide,,,,,,,
MSSHEGGKK,HLA-A03:01,MSSHEGGKK,MSSHEGGKK,0.0754,2.2719,0.3113,2.3594
SHEGGKKKA,HLA-A01:01,SHEGGKKKA,SHEGGKKKA,0.0002,41.1667,0.0133,89.0327
HEGGKKKAL,HLA-A26:01,HEGGKKKAL,HEGGKKKAL,0.001,14.6818,0.0165,60.3989
EGGKKKALK,HLA-A03:01,EGGKKKALK,EGGKKKALK,0.0014,12.034,0.0564,28.0659
GGKKKALKQ,HLA-A03:01,GGKKKALKQ,GGKKKALKQ,0.0002,24.48,0.0244,67.2066
GKKKALKQP,HLA-A03:01,GKKKALKQP,GKKKALKQP,0.0,68.75,0.0063,96.819
KKKALKQPK,HLA-A03:01,KKKALKQPK,KKKALKQPK,0.0055,7.1007,0.0838,16.5062
ALKQPKKQA,HLA-A03:01,ALKQPKKQA,ALKQPKKQA,0.0034,8.6334,0.0479,34.5175


In [30]:

def parse_netmhcpan_header(df_columns: pd.core.indexes.multi.MultiIndex):
    """
    Reads and properly parses the headers for outputs of NetMHCpan
    """
    level_0 = df_columns.get_level_values(0).tolist()
    level_1 = df_columns.get_level_values(1).tolist()
    value = 'base'
    for i, (l0, l1) in enumerate(zip(level_0, level_1)):
        if l0.find('HLA') != -1:
            value = l0
        if l1.find('Ave') != -1:
            value = 'end'
        level_0[i] = value

    return pd.MultiIndex.from_tuples([(x, y) for x, y in zip(level_0, level_1)])


def return_columns(row, df):
    """
    Returns the columns with HLA in it for multi indexing of netmhcpan xls df
    """
    return [x for x in df.columns if x[0] == row['HLA']]


def filter_rank(df_netmhcpan, which_rank):
    """
    From the df_netmhcpan, filter the df using the rank given by `which_rank`,
    Finds the minimum rank for the given `which_rank` and its corresponding HLA among all HLA results

    """
    hlas = set([x for x in df_netmhcpan.columns.get_level_values(0) if 'hla' in x.lower()])
    ranks = [x for x in df_netmhcpan.columns if x[0] in hlas and x[1].lower() == which_rank.lower()]
    df_out = pd.merge(df_netmhcpan[ranks].idxmin(axis=1).apply(lambda x: x[0]).rename('HLA'),
                      df_netmhcpan[ranks].min(axis=1).rename('tmp'),
                      left_index=True, right_index=True)
    return df_out


def get_filtered_df(df_out, df_netmhcpan):
    """
    From the output df returned by filter_rank, filters the original NetMHCpan xls df and
    keep only the values for the best-binding HLA.
    """
    # Filters the original df
    filtered = df_out.apply(lambda x: df_netmhcpan.loc[x.name, return_columns(x, df_netmhcpan)].values, axis=1)
    # reshapes the filtered df
    df_values = pd.DataFrame.from_dict(dict(zip(filtered.index, filtered.values))).T
    df_values.index.name = 'Peptide'
    df_values.columns = ['core', 'icore', 'EL_score', 'EL_rank', 'BA_score', 'BA_rank']

    return df_out.drop(columns = ['tmp']).merge(df_values, left_index=True, right_index=True)



In [37]:
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import parse_netmhcpan_header
df = pd.read_csv('../29165_NetMHCpan.xls', header=[0,1], sep = '\t')
df = df.drop(index = df.sample(3).index)
df.columns = parse_netmhcpan_header(df.columns)
pep = pd.read_csv('../output/test_pep/9mers_Human_split_0.pep', header = None)
pep.columns = ['Peptide']
df.set_index(('base', 'Peptide'), inplace=True)
df.index.name = 'Peptide'

In [38]:
df_output = filter_rank(df, 'el_rank')


In [45]:
get_filtered_df(df_output, df).to_csv('../results.csv')

In [44]:
strx='../output/test_pep/9mers_Human_split_0.pep'
strx[strx.rfind('/')+1:strx.find('.pep')]

'9mers_Human_split_0'

In [47]:
pd.read_csv(os.path.join(os.getcwd(), '../output/test_pep/9mers_Human_split_0.pep'))

,MSSHEGGKK
0,SSHEGGKKK
1,SHEGGKKKA
2,HEGGKKKAL
3,EGGKKKALK
4,GGKKKALKQ
5,GKKKALKQP
6,KKKALKQPK
7,KKALKQPKK
8,KALKQPKKQ
9,ALKQPKKQA
